In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE


import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import pandas as pd
import numpy as np
import re

In [24]:
import pandas as pd

In [25]:
recalibrate_df = pd.read_csv('recalibrate')


In [26]:
def OHE(df):
    encoder = OneHotEncoder()
    encoded_data = encoder.fit_transform(df)
    output_names = encoder.get_feature_names_out(df.columns)
    df_codified = pd.DataFrame(encoded_data.toarray(), columns=output_names, index= df.index)
    return df_codified, encoder

In [27]:
def inverse_OHE(encoded_df, encoder):
     # Realizar la inversión de la codificación one-hot
    inverse_data = encoder.inverse_transform(encoded_df)
    return inverse_data

     

In [28]:
def r_forests(X_train,X_test,y_train):
    
    # Definir el pipeline
    pipeline = Pipeline([
    ('smote', SMOTE()), # Aplicar SMOTE
    ('classifier', RandomForestClassifier()) # Modelo de clasificación
    ])
    
    param_grid = {
        'smote__sampling_strategy': ['auto'],  # Estrategia de sobremuestreo para SMOTE
        'classifier__n_estimators': [50, 100, 200],  # Número de árboles en el bosque
        'classifier__max_depth': [None, 10, 20]  # Profundidad máxima de los árboles
    }
    
    grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    best_model.fit(X_train, y_train)
    best_score= grid_search.best_score_
    print(best_score)

    y_predict = best_model.predict(X_test)

    # model.fit(X_train,y_train)
    # y_predict = model.predict(X_test)
    return y_predict

In [29]:

def model_imputate(df, column):
    # df = original_df.copy()
    # df = df.drop(columns=cols)

    #después agregar o modificar el vehicule type por el elemento a analizar. IMPORTANTING

    # Creamos dos dataframes, el primero con los valores nulos (los que queremos predecir) y el segundo con los elementos que no son nulos (que nos servirán para entrenar al modelo)
    predict_df = df[df[column].isnull()]
    train_df = df[~df[column].isnull()]

    # Seleccionamos las columnas que se clasificaran en OHE
    cols = ['registration_year','power','mileage','registration_month','brand']
    OHE_df, encoder1 = OHE(df[cols])

    # Definimos elementos y objetivos del dataframe de predición
    y_val =  predict_df[column]
    X_val = OHE_df.query('index in @predict_df.index')

    # Definimos elementos y objetivos del dataframe de entrenamiento
    y = train_df[column]
    X = OHE_df.query('index in @train_df.index')

    # Ejecutamos RandomForest para recibir la predicción
    y_predict = r_forests(X, X_val, y)

    return y_predict

   
    # report = accuracy_score(y_test,y_predict_serie)
    
    # print(report)
    
    # expected_prediction = model.predict(X_val)
    # expected_report = classification_report(y_train, y_val)
    # print(report, end= '\n\n')
    # print(expected_report)
    # return pred

    # return y_predict_categories, y_test
    # return y_predict_inverse


In [30]:
r = model_imputate(recalibrate_df, 'vehicle_type')

In [ ]:
r